In [1]:
#import Pkg; Pkg.add("Yao")

#!/usr/bin/env julia

using Yao
using Yao.ConstGate # needed for P1 = 0.5*(I - sigma_z) block

# Task 2

In [4]:
using StatsBase


#=
H(t) = Ω(t) ∑_i σ_i^x - δ(t) ∑_i n_i + u ∑_ij n_i n_j
=#

const u = 1.35
const Ω_max = 1.89
const δ_0 = -1.0
const δ_max = 1.0

function get_edges(graph::Vector{NTuple{2, Float64}})
    Nv = size(graph)[1]
    edges = falses(Nv, Nv)
    for i in 1:(Nv-1)
        xi, yi = graph[i]
        for j in (i+1):Nv
            xj, yj = graph[j]

            dij = sqrt((xi - xj)^2. + (yi - yj)^2.)
            if dij <= 1.0
                edges[i,j] = true
            end
        end
    end
    return findall(edges)
end

function Ω(t::Float64)
    if 0 <= t <= 0.25
        return (Ω_max / 0.25) * t
    elseif 0.25 < t <= 0.69
        return Ω_max
    elseif 0.69 < t <= 1
        return - Ω_max * t / 0.31 + Ω_max * (1 + 0.69/0.31)
    end
end

function δ(t::Float64)
    slope = (δ_0 - δ_max)/(0.25 - 0.69)
    if 0 <= t <= 0.25
        return δ_0
    elseif 0.25 < t <= 0.69
        return t * slope + (δ_max - slope * 0.69)
    elseif 0.69 < t <= 1
        return δ_max
    end
end 

function hamiltonian(graph::Vector{NTuple{2, Float64}}, edges::Vector{CartesianIndex{2}}, t::Float64)
    # the UD-MIS Hamiltonian
    Nv = size(graph)[1] # number of vertices

    interaction_term = map(1:size(edges)[1]) do i
        l,m = edges[i][1], edges[i][2]
        repeat(Nv,u*P1,(l,m))
    end |> sum
    interaction_term - δ(t)*sum(map(i->put(Nv,i=>P1), 1:Nv)) + Ω(t)*sum(map(i->put(Nv,i=>X), 1:Nv))
end

function run_annealing(graph::Vector{NTuple{2, Float64}}, edges::Vector{CartesianIndex{2}}, dt::Float64)
    psi_t = zero_state(size(graph)[1])
    for t in 0:dt:1.0
        h = hamiltonian(graph, edges, t)
        psi_t = psi_t |> TimeEvolution(h, dt * 100)
    end
    return psi_t
end

graph = [(0.3461717838632017, 1.4984640297338632), 
         (0.6316400411846113, 2.5754677320579895), 
         (1.3906262250927481, 2.164978861396621), 
         (0.66436005100802, 0.6717919819739032), 
         (0.8663329771713457, 3.3876341010035995), 
         (1.1643107343501296, 1.0823066243402013)
        ]
edges = get_edges(graph)
dt = 0.001


psi = run_annealing(graph, edges, dt)
open("task2_data.dat","w") do io
    for sample in measure(psi; nshots=10_000)
        println(io, sample)
    end
end

#samples = measure(psi; nshots=10)
#@show samples

In [5]:
measure(psi; nshots=10_000)

10000-element Vector{BitBasis.BitStr64{6}}:
 110100 ₍₂₎
 010101 ₍₂₎
 110100 ₍₂₎
 111100 ₍₂₎
 011100 ₍₂₎
 111100 ₍₂₎
 110100 ₍₂₎
 011100 ₍₂₎
 110100 ₍₂₎
 011100 ₍₂₎
 010101 ₍₂₎
 000101 ₍₂₎
 011100 ₍₂₎
          ⋮
 011100 ₍₂₎
 010101 ₍₂₎
 110100 ₍₂₎
 011100 ₍₂₎
 110100 ₍₂₎
 011100 ₍₂₎
 110100 ₍₂₎
 111100 ₍₂₎
 011100 ₍₂₎
 010101 ₍₂₎
 110100 ₍₂₎
 110100 ₍₂₎

# Task 3

In [6]:
using StatsBase


#=
H(t) = Ω(t) ∑_i σ_i^x - δ(t) ∑_i n_i + u ∑_ij n_i n_j
=#

const u = 1.35
const Ω_max = 1.89
const δ_0 = -1.0
const δ_max = 1.0

function get_edges(graph::Vector{NTuple{2, Float64}})
    Nv = size(graph)[1]
    edges = falses(Nv, Nv)
    for i in 1:(Nv-1)
        xi, yi = graph[i]
        for j in (i+1):Nv
            xj, yj = graph[j]

            dij = sqrt((xi - xj)^2. + (yi - yj)^2.)
            if dij <= 1.0
                edges[i,j] = true
            end
        end
    end
    return findall(edges)
end

function Ω(t::Float64)
    if 0 <= t <= 0.25
        return (Ω_max / 0.25) * t
    elseif 0.25 < t <= 0.69
        return Ω_max
    elseif 0.69 < t <= 1
        return - Ω_max * t / 0.31 + Ω_max * (1 + 0.69/0.31)
    end
end

function δ(t::Float64)
    slope = (δ_0 - δ_max)/(0.25 - 0.69)
    if 0 <= t <= 0.25
        return δ_0
    elseif 0.25 < t <= 0.69
        return t * slope + (δ_max - slope * 0.69)
    elseif 0.69 < t <= 1
        return δ_max
    end
end 

function hamiltonian(graph::Vector{NTuple{2, Float64}}, edges::Vector{CartesianIndex{2}}, t::Float64)
    # the UD-MIS Hamiltonian
    Nv = size(graph)[1] # number of vertices

    interaction_term = map(1:size(edges)[1]) do i
        l,m = edges[i][1], edges[i][2]
        repeat(Nv,u*P1,(l,m))
    end |> sum
    interaction_term - δ(t)*sum(map(i->put(Nv,i=>P1), 1:Nv)) + Ω(t)*sum(map(i->put(Nv,i=>X), 1:Nv))
end

function run_annealing(graph::Vector{NTuple{2, Float64}}, edges::Vector{CartesianIndex{2}}, dt::Float64)
    psi_t = zero_state(size(graph)[1])
    for t in 0:dt:1.0
        h = hamiltonian(graph, edges, t)
        psi_t = psi_t |> TimeEvolution(h, dt * 100)
    end
    return psi_t
end


graph = [(1.19, 4.25),(2.71, 3.48),(1.19, 3.51),(2, 3.38),(1.12, 2.86),(1.70, 2.42),(2.36, 2.54),
    (1.52, 1.48),(2.15, 1.54),(2.14, 1.87),(1.72, 0.86),(2.29, 0.87)]
graph=convert( Vector{Tuple{Float64, Float64}}, graph)

edges = get_edges(graph)
dt = 0.001

psi = run_annealing(graph, edges, dt)
open("task3_data.dat","w") do io
    for sample in measure(psi; nshots=10_000)
        println(io, sample)
    end
end

In [7]:
measure(psi; nshots=10_000)

10000-element Vector{BitBasis.BitStr64{12}}:
 010001010011 ₍₂₎
 000011010011 ₍₂₎
 000011010011 ₍₂₎
 101000011001 ₍₂₎
 101000010011 ₍₂₎
 100001010011 ₍₂₎
 011000010011 ₍₂₎
 101000010011 ₍₂₎
 100011010011 ₍₂₎
 100001010011 ₍₂₎
 011000010011 ₍₂₎
 100001010011 ₍₂₎
 011000010011 ₍₂₎
                ⋮
 101000010011 ₍₂₎
 011000011000 ₍₂₎
 011000010011 ₍₂₎
 000101010011 ₍₂₎
 000101010011 ₍₂₎
 010001010011 ₍₂₎
 101000010011 ₍₂₎
 000101010011 ₍₂₎
 000010011001 ₍₂₎
 000101010011 ₍₂₎
 101000010011 ₍₂₎
 000101010011 ₍₂₎